# Sistem Pencarian Obat Berdasarkan Keluhan dan Nama Obat Berbasis Information Retrieval
*Tugas Akhir Mata Kuliah Sistem Temu Kembali Informasi*

Notebook ini mencakup seluruh tahapan:
- Data Collection (Corpus Generation)  
- Preprocessing & Feature Extraction  
- Query Processing & Search System  
- Evaluation & Testing  

### Setup & Import Libraries

In [1]:
import os
import re
import json
import pickle
import math
from pathlib import Path
from collections import defaultdict, Counter
import pandas as pd
import numpy as np

### Sastrawi untuk Bahasa Indonesia

In [2]:
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
    SASTRAWI_AVAILABLE = True
    print("Sastrawi available")
except ImportError:
    SASTRAWI_AVAILABLE = False
    print("Sastrawi not installed. Install with: pip install Sastrawi")

print("Libraries imported")

Sastrawi available
Libraries imported


### Load Corpus

In [3]:
def load_corpus(corpus_dir='corpus'):
    """Load all documents from corpus"""
    documents = {}
    corpus_path = Path(corpus_dir)
    files = sorted(corpus_path.glob('OBT*.txt'))
    
    for filepath in files:
        doc_id = filepath.stem
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        documents[doc_id] = content
    
    return documents

# Load corpus
documents = load_corpus('corpus')
print(f"Loaded {len(documents)} documents")

# Preview
sample_doc_id = list(documents.keys())[0]
print(f"\nSample Document: {sample_doc_id}")
print("="*80)
print(documents[sample_doc_id][:500])
print("...")

Loaded 100 documents

Sample Document: OBT001
DOKUMEN OBAT ID: OBT001

INFORMASI PRODUK
----------------
Nama Obat       : Panadol 500mg
Nama Generik    : Paracetamol
Golongan Obat   : Analgesik Antipiretik
Bentuk Sediaan  : Tablet
Produsen        : GlaxoSmithKline

KOMPOSISI
---------
Paracetamol 500mg

INDIKASI DAN KEGUNAAN
---------------------
Meredakan demam dan nyeri ringan hingga sedang seperti sakit kepala sakit gigi nyeri haid nyeri otot dan menurunkan
...


In [4]:
# Load Metadata
with open('metadata/obat_metadata.json', 'r', encoding='utf-8') as f:
    metadata_list = json.load(f)

metadata_df = pd.DataFrame(metadata_list)
print(f"Loaded {len(metadata_df)} metadata records")

# Display sample
metadata_df.head()

Loaded 100 metadata records


,id,nama_obat,nama_generik,golongan,komposisi,bentuk_sediaan,indikasi,dosis,efek_samping,kontraindikasi,harga_min,harga_max,perlu_resep,produsen,tags
0,OBT001,Panadol 500mg,Paracetamol,Analgesik Antipiretik,Paracetamol 500mg,Tablet,Meredakan demam dan nyeri ringan hingga sedang...,Dewasa dan anak di atas 12 tahun: 1-2 tablet s...,Mual muntah ruam kulit reaksi alergi kerusakan...,Gangguan fungsi hati berat hipersensitif terha...,5000,15000,Tidak,GlaxoSmithKline,demam;sakit kepala;nyeri;panas;flu;sakit gigi;...
1,OBT002,Biogesic 500mg,Paracetamol,Analgesik Antipiretik,Paracetamol 500mg,Tablet,Menurunkan demam dan meredakan nyeri ringan hi...,Dewasa: 1-2 tablet 3-4 kali sehari. Diminum se...,Reaksi alergi kulit mual gangguan fungsi hati ...,Gangguan fungsi hati berat hipersensitif parac...,4000,12000,Tidak,Combiphar,demam;panas;sakit kepala;nyeri;flu
2,OBT003,Bodrex Tablet,Paracetamol + Caffeine,Analgesik,Paracetamol 500mg Caffeine 50mg,Tablet,Meringankan sakit kepala dan sakit gigi dengan...,Dewasa: 1-2 tablet 3 kali sehari sesudah makan,Jantung berdebar gelisah insomnia tremor mual,Gangguan jantung tukak lambung aktif insomnia ...,3000,8000,Tidak,Tempo Scan Pacific,sakit kepala;migrain;pusing;sakit gigi
3,OBT004,Paramex Tablet,Paracetamol + Propyphenazone,Analgesik,Paracetamol 250mg Propyphenazone 150mg Caffein...,Tablet,Meredakan sakit kepala sakit gigi nyeri otot d...,Dewasa: 1-2 tablet 3 kali sehari setelah makan,Mual muntah pusing gangguan pencernaan reaksi ...,Gangguan fungsi hati dan ginjal berat tukak la...,4000,10000,Tidak,Konimex,sakit kepala;sakit gigi;nyeri otot;pusing
4,OBT005,Oskadon Tablet,Paracetamol + Ibuprofen,Analgesik,Paracetamol 250mg Ibuprofen 200mg Caffeine 50mg,Tablet,Meringankan sakit kepala termasuk sakit kepala...,Dewasa: 1 tablet 3 kali sehari sesudah makan,Mual muntah nyeri lambung pusing mengantuk,Tukak lambung atau duodenum gangguan ginjal da...,5000,12000,Tidak,Bayer,sakit kepala;migrain;sakit gigi;nyeri


### Statistics

In [5]:
print("Dataset Statistics:")
print(f"Total obat: {len(metadata_df)}")
print(f"\nDistribusi Kebutuhan Resep:")
print(metadata_df['perlu_resep'].value_counts())

print(f"\nHarga:")
print(f"Min: Rp {metadata_df['harga_min'].min():,}")
print(f"Max: Rp {metadata_df['harga_max'].max():,}")
print(f"Mean: Rp {metadata_df['harga_min'].mean():,.0f} - Rp {metadata_df['harga_max'].mean():,.0f}")

# Top golongan
print(f"\nTop 10 Golongan Obat:")
print(metadata_df['golongan'].value_counts().head(10))

Dataset Statistics:
Total obat: 100

Distribusi Kebutuhan Resep:
perlu_resep
Ya       51
Tidak    49
Name: count, dtype: int64

Harga:
Min: Rp 2,000
Max: Rp 180,000
Mean: Rp 13,490 - Rp 31,480

Top 10 Golongan Obat:
golongan
Analgesik                       3
Multivitamin                    3
Kortikosteroid                  3
Antikonvulsan                   3
Angiotensin Receptor Blocker    3
Analgesik Antipiretik           2
Suplemen Herbal                 2
Antigout                        2
Calcium Channel Blocker         2
Benzodiazepin Anxiolytic        2
Name: count, dtype: int64


### Build Text Preprocessor

In [ ]:
class TextPreprocessor:
    """Preprocessing text untuk Bahasa Indonesia"""
    
    def __init__(self):
        if SASTRAWI_AVAILABLE:
            self.stemmer = StemmerFactory().create_stemmer()
            self.stopword_remover = StopWordRemoverFactory().create_stop_word_remover()
            self.custom_stopwords = {
                'mg', 'ml', 'tablet', 'kapsul', 'sirup', 'gram', 'yang', 'pada', 'untuk',
                'dengan', 'dari', 'atau', 'dan', 'seperti', 'termasuk', 'dapat', 'akan'
            }
        else:
            self.stemmer = None
            self.stopword_remover = None
            self.custom_stopwords = set()
        
        print("TextPreprocessor initialized")
    
    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r'\b\d+\b', '', text)
        text = re.sub(r'[^a-z\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        return text.split()
    
    def remove_stopwords(self, tokens):
        if not SASTRAWI_AVAILABLE:
            basic_stopwords = {'yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'dengan', 'untuk'}
            return [t for t in tokens if t not in basic_stopwords and t not in self.custom_stopwords]
        
        text = ' '.join(tokens)
        text = self.stopword_remover.remove(text)
        tokens = text.split()
        return [t for t in tokens if t not in self.custom_stopwords and len(t) > 2]
    
    def stem(self, tokens):
        if not SASTRAWI_AVAILABLE or not self.stemmer:
            return tokens
        return [self.stemmer.stem(token) for token in tokens]
    
    def preprocess(self, text):
        text = self.clean_text(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        tokens = self.stem(tokens)
        tokens = [t for t in tokens if len(t) > 2]
        return tokens

# Initialize preprocessor
preprocessor = TextPreprocessor()


# Preprocess All Documents

print("Preprocessing documents...")
processed_docs = {}

for doc_id, content in documents.items():
    tokens = preprocessor.preprocess(content)
    processed_docs[doc_id] = tokens

print(f"Preprocessed {len(processed_docs)} documents")

# Preview
sample_id = list(processed_docs.keys())[0]
print(f"\n Sample tokens from {sample_id}:")
print(f"Total tokens: {len(processed_docs[sample_id])}")
print(f"First 30 tokens: {processed_docs[sample_id][:30]}")

TextPreprocessor initialized
Preprocessing documents...
Preprocessed 100 documents

 Sample tokens from OBT001:
Total tokens: 93
First 30 tokens: ['dokumen', 'obat', 'obt', 'informasi', 'produk', 'nama', 'obat', 'panadol', 'nama', 'generik', 'paracetamol', 'golong', 'obat', 'analgesik', 'antipiretik', 'bentuk', 'sedia', 'produsen', 'glaxosmithkline', 'komposisi', 'paracetamol', 'indikasi', 'guna', 'reda', 'demam', 'nyeri', 'ringan', 'hingga', 'sedang', 'sakit']


In [7]:
# Build vocabulary
all_tokens = []
for tokens in processed_docs.values():
    all_tokens.extend(tokens)

token_freq = Counter(all_tokens)

print(f"Vocabulary Statistics:")
print(f"Total tokens: {len(all_tokens)}")
print(f"Unique tokens: {len(token_freq)}")
print(f"Average tokens per document: {len(all_tokens) / len(processed_docs):.1f}")

print(f"\n🔝 Top 20 most frequent tokens:")
for token, freq in token_freq.most_common(20):
    print(f"   {token}: {freq}")

Vocabulary Statistics:
Total tokens: 8305
Unique tokens: 776
Average tokens per document: 83.0

🔝 Top 20 most frequent tokens:
   obat: 628
   nama: 299
   guna: 207
   informasi: 200
   harga: 200
   dokumen: 199
   cari: 199
   generik: 151
   hari: 113
   gejala: 108
   produsen: 107
   dosis: 106
   butuh: 104
   efek: 103
   bentuk: 102
   kandung: 101
   obt: 100
   produk: 100
   golong: 100
   sedia: 100


### TF-IDF Vectorization

In [8]:
class TFIDFVectorizer:
    """TF-IDF Vectorizer untuk IR System"""
    
    def __init__(self):
        self.vocabulary = {}
        self.idf = {}
        self.doc_vectors = {}
        self.doc_lengths = {}
        
    def fit_transform(self, documents):
        print("\n Building TF-IDF vectors...")
        
        # Build vocabulary and calculate document frequency
        term_doc_count = defaultdict(int)
        all_terms = set()
        
        for doc_id, tokens in documents.items():
            unique_terms = set(tokens)
            for term in unique_terms:
                term_doc_count[term] += 1
            all_terms.update(tokens)
        
        # Create vocabulary mapping
        self.vocabulary = {term: idx for idx, term in enumerate(sorted(all_terms))}
        
        # Calculate IDF
        N = len(documents)
        for term, df in term_doc_count.items():
            self.idf[term] = math.log10(N / df)
        
        print(f"   Vocabulary size: {len(self.vocabulary)}")
        print(f"   Total documents: {N}")
        
        # Calculate TF-IDF for each document
        for doc_id, tokens in documents.items():
            tf = Counter(tokens)
            doc_length = len(tokens)
            
            tfidf_vector = {}
            magnitude = 0
            
            for term, count in tf.items():
                if term in self.vocabulary and term in self.idf:
                    tf_value = count / doc_length
                    tfidf = tf_value * self.idf[term]
                    
                    term_id = self.vocabulary[term]
                    tfidf_vector[term_id] = tfidf
                    magnitude += tfidf ** 2
            
            self.doc_vectors[doc_id] = tfidf_vector
            self.doc_lengths[doc_id] = math.sqrt(magnitude)
        
        print(f"TF-IDF vectors created for {len(self.doc_vectors)} documents")
        
        return self.doc_vectors
    
    def transform_query(self, query_tokens):
        tf = Counter(query_tokens)
        query_length = len(query_tokens)
        
        tfidf_vector = {}
        magnitude = 0
        
        for term, count in tf.items():
            if term in self.vocabulary and term in self.idf:
                tf_value = count / query_length
                tfidf = tf_value * self.idf[term]
                
                term_id = self.vocabulary[term]
                tfidf_vector[term_id] = tfidf
                magnitude += tfidf ** 2
        
        return tfidf_vector, math.sqrt(magnitude)

# Build TF-IDF
vectorizer = TFIDFVectorizer()
doc_vectors = vectorizer.fit_transform(processed_docs)



 Building TF-IDF vectors...
   Vocabulary size: 776
   Total documents: 100
TF-IDF vectors created for 100 documents


In [9]:
# Analyze TF-IDF

# Find terms with highest IDF (rare terms)
idf_sorted = sorted(vectorizer.idf.items(), key=lambda x: x[1], reverse=True)

print(" Top 10 terms with highest IDF (rare terms):")
for term, idf_value in idf_sorted[:10]:
    print(f"   {term}: {idf_value:.4f}")

print("\n Top 10 terms with lowest IDF (common terms):")
for term, idf_value in idf_sorted[-10:]:
    print(f"   {term}: {idf_value:.4f}")

 Top 10 terms with highest IDF (rare terms):
   glaxosmithkline: 2.0000
   panadol: 2.0000
   rusa: 2.0000
   vaksinasi: 2.0000
   biogesic: 2.0000
   efektivitas: 2.0000
   bodrex: 2.0000
   propyphenazone: 2.0000
   konimex: 2.0000
   paramex: 2.0000

 Top 10 terms with lowest IDF (common terms):
   guna: 0.0000
   bentuk: 0.0000
   ingat: 0.0000
   sedia: 0.0000
   dokumen: 0.0000
   kontraindikasi: 0.0000
   pakai: 0.0000
   produsen: 0.0000
   butuh: 0.0000
   kata: 0.0000


### Build Inverted Index

In [10]:
class InvertedIndex:
    """Inverted Index untuk efficient retrieval"""
    
    def __init__(self):
        self.index = defaultdict(list)
        
    def build(self, doc_vectors, vocabulary):
        print("\nBuilding Inverted Index...")
        
        id_to_term = {idx: term for term, idx in vocabulary.items()}
        
        for doc_id, vector in doc_vectors.items():
            for term_id, tfidf in vector.items():
                term = id_to_term[term_id]
                self.index[term].append((doc_id, tfidf))
        
        # Sort by tfidf descending
        for term in self.index:
            self.index[term].sort(key=lambda x: x[1], reverse=True)
        
        print(f"Inverted Index built with {len(self.index)} terms")
        
        return self.index



In [11]:
# Build inverted index
inv_index = InvertedIndex()
inverted_index = inv_index.build(doc_vectors, vectorizer.vocabulary)

# Sample inverted index
sample_terms = list(inverted_index.keys())[:5]
print(f"\nSample Inverted Index:")
for term in sample_terms:
    print(f"\nTerm: '{term}'")
    print(f"   Appears in {len(inverted_index[term])} documents")
    print(f"   Top 3 documents: {inverted_index[term][:3]}")


Building Inverted Index...
Inverted Index built with 776 terms

Sample Inverted Index:

Term: 'dokumen'
   Appears in 100 documents
   Top 3 documents: [('OBT001', 0.0), ('OBT002', 0.0), ('OBT003', 0.0)]

Term: 'obat'
   Appears in 100 documents
   Top 3 documents: [('OBT001', 0.0), ('OBT002', 0.0), ('OBT003', 0.0)]

Term: 'obt'
   Appears in 100 documents
   Top 3 documents: [('OBT001', 0.0), ('OBT002', 0.0), ('OBT003', 0.0)]

Term: 'informasi'
   Appears in 100 documents
   Top 3 documents: [('OBT001', 0.0), ('OBT002', 0.0), ('OBT003', 0.0)]

Term: 'produk'
   Appears in 100 documents
   Top 3 documents: [('OBT001', 0.0), ('OBT002', 0.0), ('OBT003', 0.0)]


### Query Processing & Search

In [ ]:
class SearchEngine:
    """Search Engine untuk IR Obat"""
    
    def __init__(self, preprocessor, vectorizer, metadata_dict, doc_vectors, doc_magnitudes):
        self.preprocessor = preprocessor
        self.vectorizer = vectorizer
        self.metadata = metadata_dict
        
        # ini data index dokumen (wajib ada)
        self.doc_vectors = doc_vectors          # list/dict: {doc_id: sparse_vec}
        self.doc_magnitudes = doc_magnitudes    # list/dict: {doc_id: magnitude}
        
    def cosine_similarity(self, vec1, vec2, mag1, mag2):
        if mag1 == 0 or mag2 == 0:
            return 0.0
        
        dot_product = sum(
            vec1.get(term_id, 0) * vec2.get(term_id, 0)
            for term_id in set(vec1.keys()) | set(vec2.keys())
        )
        return dot_product / (mag1 * mag2)

    def search(self, query, top_k=5):
        # 1) preprocess query
        tokens = self.preprocessor.preprocess(query)

        # 2) vectorize query -> harusnya ngembaliin (sparse_vec, magnitude)
        q_vec, q_mag = self.vectorizer.transform_query(tokens)

        # 3) hitung similarity ke semua dokumen
        scored = []
        for doc_id, d_vec in self.doc_vectors.items():
            d_mag = self.doc_magnitudes.get(doc_id, 0)
            score = self.cosine_similarity(q_vec, d_vec, q_mag, d_mag)
            if score > 0:
                scored.append((doc_id, score))

        # 4) ambil top-k
        scored.sort(key=lambda x: x[1], reverse=True)
        top = scored[:top_k]

        # 5) bentuk hasil dengan metadata (kalau ada)
        results = []
        for doc_id, score in top:
            meta = self.metadata.get(doc_id, {})
            results.append({
                "doc_id": doc_id,
                "score": score,
                **meta
            })

        return results, tokens


In [ ]:
def search(self, query, top_k=10):
        # Preprocess query
        query_tokens = self.preprocessor.preprocess(query)
        
        if not query_tokens:
            return []
        
        # Transform query to TF-IDF
        query_vector, query_magnitude = self.vectorizer.transform_query(query_tokens)
        
        # Calculate similarity with all documents
        scores = []
        
        for doc_id in self.vectorizer.doc_vectors.keys():
            doc_vector = self.vectorizer.doc_vectors.get(doc_id, {})
            doc_magnitude = self.vectorizer.doc_lengths.get(doc_id, 0)
            
            similarity = self.cosine_similarity(
                query_vector, doc_vector,
                query_magnitude, doc_magnitude
            )
            
            if similarity > 0:
                scores.append((doc_id, similarity))
        
        # Sort by similarity descending
        scores.sort(key=lambda x: x[1], reverse=True)
        
        # Get top-k results with metadata
        results = []
        for doc_id, score in scores[:top_k]:
            metadata = self.metadata.get(doc_id, {})
            results.append((doc_id, score, metadata))
        
        return results, query_tokens

In [14]:
# Build metadata dict
metadata_dict = {item['id']: item for item in metadata_list}

# Initialize search engine
search_engine = SearchEngine(preprocessor, vectorizer, metadata_dict)
print("Search Engine initialized")

Search Engine initialized


### Test Search

In [16]:
def display_results(results, query_tokens):
    if not results:
        print("No results found")
        return
    
    print(f"\nQuery tokens: {query_tokens}")
    print(f"Found {len(results)} results\n")
    print("="*100)

    for rank, (doc_id, score, metadata) in enumerate(results, 1):
        print(f"\n#{rank} | Score: {score:.4f} | ID: {doc_id}")
        print(f"Nama: {metadata.get('nama_obat', 'N/A')}")
        print(f"Generik: {metadata.get('nama_generik', 'N/A')}")
        print(f"Golongan: {metadata.get('golongan', 'N/A')}")
        print(f"Indikasi: {metadata.get('indikasi', 'N/A')[:150]}...")
        print(f"Harga: Rp {metadata.get('harga_min', 0):,} - Rp {metadata.get('harga_max', 0):,}")
        print(f"Resep: {metadata.get('perlu_resep', 'N/A')}")
        print("-"*100)

### Test Case 1: Demam dan Sakit Kepala

In [ ]:
query1 = "demam sakit kepala"
results1, tokens1 = search_engine.search(query1, top_k=5)
display_results(results1, tokens1)

AttributeError: 'SearchEngine' object has no attribute 'searchengine'